Search satellite by partial satellite name:

Create `space-track-credentials.json` from `space-track-credentials_TEMPLATE.json` first


In [1]:
import requests
import pandas as pd
import os.path
import urllib
import json

In [2]:
# specific satellite name for the rest of this file for convenience
# we search by LIKE so partial match works

# satellite_name = "MICROSAT-R"
satellite_name = "USA 193"
# satellite_name = "FENGYUN 1C"
# satellite_name = "METEOR"
DEBUT_FILE_PATH = 'test/satname-debut-{}.csv'
GP_HISTORY_FILE_PATH = 'test/satname-gp-history-{}.csv'

In [3]:
columns = ["NORAD_CAT_ID","EPOCH","MEAN_MOTION","ECCENTRICITY","INCLINATION","RA_OF_ASC_NODE","ARG_OF_PERICENTER","MEAN_ANOMALY","EPHEMERIS_TYPE","CLASSIFICATION_TYPE","ELEMENT_SET_NO","REV_AT_EPOCH","BSTAR","MEAN_MOTION_DOT","MEAN_MOTION_DDOT","SEMIMAJOR_AXIS","PERIOD","APOAPSIS","PERIAPSIS"]

In [4]:
if "auth_cookie" not in locals():
    auth_cookie = None

In [31]:
len(auth_cookie)

1

In [32]:
len(None)

TypeError: object of type 'NoneType' has no len()

In [25]:
auth_cookie = space_track_login()

Logging in space-track.org with identity: ttcchen@umich.edu
Logged in


In [35]:
if type(auth_cookie) == requests.cookies.RequestsCookieJar:
    auth_cookie.clear_expired_cookies()
if auth_cookie == None or len(auth_cookie) == 0:
    print(f"Auth expired.")
    auth_cookie = space_track_login()

Auth expired.
Logging in space-track.org with identity: ttcchen@umich.edu
Logged in


In [22]:
# import time

# current_time = time.time()
# print(current_time)
# # print(auth_cookie.expires)
# auth_cookie.clear_expired_cookies()
# auth_cookie

In [6]:
def space_track_login():
    with open("space-track-credentials.json") as json_file:
        credentials = json.load(json_file)
    print("Logging in space-track.org with identity:",credentials['identity'])
    url = "https://www.space-track.org/ajaxauth/login"
    x = requests.post(url, data = credentials)
    if x.status_code == 200:
        print("Logged in")
        return x.cookies
    else:
        print("Failed with code:",x.status_code)
        return False
    
def get_cookie():
    global auth_cookie
    if auth_cookie == None:
        auth_cookie = space_track_login()
    return auth_cookie

In [7]:
def get_satellite_debut(satname):
    csvurl="https://www.space-track.org/basicspacedata/query/class/satcat_debut/SATNAME/~~{}/orderby/INTLDES%20asc/format/csv/metadata/true/".format(satname)
    print("Downloading debut for satname LIKE: ",satname,"\nURL:",csvurl)
    r = requests.get(csvurl, allow_redirects=True, cookies=get_cookie())
    open(DEBUT_FILE_PATH.format(satname), 'wb').write(r.content)
    print("Saved file:",DEBUT_FILE_PATH.format(satname))
    
def get_norad_id_for_satname(satname):
    if os.path.exists(DEBUT_FILE_PATH.format(satname)):
        print("Load from disk:", DEBUT_FILE_PATH.format(satname))
        pass
    else:
        get_satellite_debut(satname)
    df = pd.read_csv(DEBUT_FILE_PATH.format(satname))
    return df.NORAD_CAT_ID.values

def get_satellite_gp_history(norad_ids,satname):
    s = ",".join([str(x) for x in norad_ids])
    csvurl="https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{}/orderby/CCSDS_OMM_VERS asc/format/csv/emptyresult/show".format(s)
    print("Downloading gp history for satname LIKE: ",satname,"\nURL:",csvurl)
    r = requests.get(csvurl, allow_redirects=True, cookies=get_cookie())
    open(GP_HISTORY_FILE_PATH.format(satname), 'wb').write(r.content)
    print("Saved file:",GP_HISTORY_FILE_PATH.format(satname))
    



In [21]:
get_satellite_gp_history(get_norad_id_for_satname(satellite_name),satellite_name)

Load from disk: test/satname-debut-USA 193.csv
URL: https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/29651,32524,32525,32526,32527,32528,32529,32530,32531,32532,32533,32534,32535,32536,32537,32538,32539,32540,32541,32542,32543,32544,32545,32546,32578,32579,32580,32581,32582,32583,32584,32585,32586,32587,32588,32589,32590,32591,32592,32593,32594,32595,32596,32597,32598,32599,32600,32601,32602,32502,32603,32604,32605,32606,32607,32608,32609,32610,32611,32612,32613,32614,32615,32616,32617,32618,32619,32620,32621,32622,32623,32624,32625,32626,32503,32627,32628,32629,32630,32631,32632,32633,32634,32635,32636,32637,32638,32639,32640,32641,32642,32643,32644,32645,32646,32647,32648,32649,32650,32504,32651,32652,32653,32654,32655,32656,32657,32658,32659,32660,32661,32662,32663,32664,32665,32666,32667,32668,32669,32670,32671,32672,32673,32674,32505,32675,32676,32677,32678,32679,32680,32681,32682,32683,32684,32685,32688,32689,32690,32691,32692,32693,32694,32695,32696

In [8]:
s = get_norad_id_for_satname(satellite_name)

Load from disk: test/satname-debut-USA 193.csv


In [29]:
df = pd.read_csv("test/satname-gp-history-USA 193.csv")

In [35]:
df[columns].to_pickle("test/satname-gp-history-USA 193.pkl.gz","gzip")

In [36]:
df2 = pd.read_pickle("test/satname-gp-history-USA 193.pkl.gz","gzip")

In [37]:
df2

,NORAD_CAT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS
0,32502,2008-02-26T06:28:42.994847,14.276680,0.088170,57.2436,6.9378,62.6549,306.6959,0,U,999,73,0.005631,0.059321,0.000008,7177.677,100.863,1432.397,166.688
1,32502,2008-02-26T06:28:42.994847,14.276680,0.088170,57.2436,6.9378,62.6549,306.6959,0,U,999,73,0.005631,0.059321,0.000008,7177.677,100.863,1432.397,166.688
2,32502,2008-02-27T00:54:07.240320,14.399764,0.083620,57.2262,4.1353,63.6059,310.7238,0,U,999,84,0.005611,0.074468,0.000008,7136.717,100.001,1355.354,161.811
3,32502,2008-02-27T05:52:03.042624,14.448921,0.081356,57.2555,3.3614,64.0891,305.2461,0,U,999,87,0.007824,0.100038,0.000008,7120.522,99.661,1321.682,163.092
4,32502,2008-02-28T08:13:35.129280,14.741211,0.069074,57.2641,359.2098,66.5334,309.1735,0,U,999,103,0.012348,0.176939,0.000009,7026.084,97.685,1133.270,162.628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,35425,2009-09-26T15:34:49.394784,15.367304,0.034527,56.1183,6.0557,29.7170,329.6798,0,U,999,6078,0.000935,0.003185,0.000009,6833.926,93.705,691.748,219.835
9746,35425,2009-10-03T14:23:40.277471,15.412508,0.032428,56.1172,335.5630,45.4257,317.8063,0,U,999,6185,0.001009,0.003443,0.000038,6820.557,93.430,663.600,221.245
9747,35425,2009-10-27T08:50:46.588128,16.042014,0.009871,56.1206,227.2647,106.1540,254.9798,0,U,999,6551,0.002313,0.029832,0.000011,6640.939,89.764,328.354,197.254
9748,35425,2009-10-28T10:13:06.377663,16.118964,0.007808,56.1122,222.2142,88.7731,273.8156,0,U,999,6568,0.002059,0.037788,0.000011,6619.787,89.335,293.336,189.967
